In [1]:
import AlGDock.BindingPMF_plots
from AlGDock.BindingPMF_plots import *
import os, shutil, glob

phases = ['NAMD_Gas', 'NAMD_OBC']

self = AlGDock.BindingPMF_plots.BPMF_plots(\
  dir_dock='dock', dir_cool='cool',\
  ligand_database='prmtopcrd/ligand.db', \
  forcefield='prmtopcrd/gaff2.dat', \
  ligand_prmtop='prmtopcrd/ligand.prmtop', \
  ligand_inpcrd='prmtopcrd/ligand.trans.inpcrd', \
  ligand_mol2='prmtopcrd/ligand.mol2', \
  ligand_rb='prmtopcrd/ligand.rb', \
  receptor_prmtop='prmtopcrd/receptor.prmtop', \
  receptor_inpcrd='prmtopcrd/receptor.trans.inpcrd', \
  receptor_fixed_atoms='prmtopcrd/receptor.pdb', \
  complex_prmtop='prmtopcrd/complex.prmtop', \
  complex_inpcrd='prmtopcrd/complex.trans.inpcrd', \
  complex_fixed_atoms='prmtopcrd/complex.pdb', \
  score = 'prmtopcrd/anchor_and_grow_scored.mol2', \
  temperature_scaling = 'Quadratic', \
  pose = -1, \
  rmsd=True, \
  dir_grid='grids', \
  protocol='Adaptive', cool_therm_speed=25.0, dock_therm_speed=0.25, \
  T_HIGH=450.0, T_SIMMIN=300.0, T_TARGET=300.0, \
  sampler='HMC', \
  MCMC_moves=1, \
  sampling_importance_resampling = True, \
  solvation = 'Full', \
  seeds_per_state=10, steps_per_seed=200, darts_per_seed=0, \
  sweeps_per_cycle=50, snaps_per_cycle=20, attempts_per_sweep=100, \
  steps_per_sweep=50, darts_per_sweep=0, \
  cool_repX_cycles=3, dock_repX_cycles=4, \
  site='Sphere', site_center=[1.7416, 1.7416, 1.7416], \
  site_max_R=0.6, \
  site_density=10., \
  phases=phases, \
  cores=-1, \
  random_seed=-1, \
  max_time=240, \
  keep_intermediate=True)

###########
# AlGDock #
###########
Molecular docking with adaptively scaled alchemical interaction grids

in /Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/AlGDock/BindingPMF.py
last modified Thu Aug  3 10:05:48 2017
    
using 4/4 available cores
using random number seed of -1
  no progress information for dock

*** Directories ***
  start: /Users/dminh/Installers/AlGDock-0.0.1/Example
  cool: /Users/dminh/Installers/AlGDock-0.0.1/Example/cool
  dock: /Users/dminh/Installers/AlGDock-0.0.1/Example/dock

*** Files ***
previously stored in cool directory:
  prmtop:
    L: prmtopcrd/ligand.prmtop
  ligand_database: prmtopcrd/ligand.db
  forcefield: ../Data/gaff2.dat
  inpcrd:
    L: prmtopcrd/ligand.trans.inpcrd

from arguments and defaults:
  ligand_database: prmtopcrd/ligand.db
  forcefield: ../Data/gaff2.dat
  prmtop:
    L: prmtopcrd/ligand.prmtop
    R: prmtopcrd/receptor.prmtop
    RL: prmtopcrd/complex.prmtop
  inpcrd:
    L: prmtopcrd/ligand.trans.i

In [2]:
self._run('all')


>>> Initial docking, starting at Thu, 03 Aug 2017 10:06:19

  LJr grid loaded from LJr.nc in 0.43 s
  LJa grid loaded from LJa.nc in 0.18 s
  ELE grid loaded from pbsa.nc in 0.03 s
  minimized 0 configurations in 0.00 s
  the first 0 energies are:
  
  keeping 0 minimized configurations out of
  0 from xtal, 692 from dock6, 0 from initial docking, and 0 duplicated
  allocated memory for interaction energies
  allocated memory for 10 translations
  10 ligand configurations were randomly docked into the binding site using 10 translations and 100 rotations 
  the predicted free energy difference between the first and second docking states is 0 (0)
  raveled energy terms in 0.00 s
  random docking complete in 4.83 s
  sLJr grid loaded from LJr.nc in 0.36 s
  sELE grid loaded from pbsa.nc in 0.03 s
  at a=5.000e-02 in 1.35 s: ExternalMC 4/50=0.08 (0.0 s); Sampler 6/40=0.15 (0.4 s); 
    dt=2.00 fs; tL_tensor=8.129e+04
  at a=5.000e-02 in 0.89 s: ExternalMC 3/50=0.06 (0.0 s); Sampler 11/40=

/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/scipy/optimize/minpack.py:237: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


  calculated grid scaling free energy of 1543.48 RT using cycles 1 to 1
  wrote to f_RL.pkl.gz


/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/scipy/optimize/minpack.py:237: RuntimeWarning: xtol=0.000000 is too small, no further improvement in the approximate
  solution is possible.
  warnings.warn(msg, RuntimeWarning)
/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/AlGDock/BindingPMF.py:3024: RuntimeWarning: overflow encountered in exp
  acc = np.exp(-u_kln[0,1,:N]-u_kln[1,0,:N]+u_kln[0,0,:N]+u_kln[1,1,:N])


  inserted state between 0 and 0.05 at 0.025
  to improve acceptance rate from 0 to 0.3
  inserted state between 0 and 0.025 at 0.0125
  to improve acceptance rate from 0.3 to 0.3
Error with BAR. Using FEP.


/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/pymbar-3.0.0.dev0-py2.7-macosx-10.5-x86_64.egg/pymbar/bar.py:264: RuntimeWarning: invalid value encountered in double_scalars
  DeltaF = UpperB - FUpperB * (UpperB - LowerB) / (FUpperB - FLowerB)
/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/pymbar-3.0.0.dev0-py2.7-macosx-10.5-x86_64.egg/pymbar/bar.py:119: RuntimeWarning: invalid value encountered in less
  max_arg_F = np.choose(np.less(0.0, exp_arg_F), (0.0, exp_arg_F))
/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/pymbar-3.0.0.dev0-py2.7-macosx-10.5-x86_64.egg/pymbar/bar.py:136: RuntimeWarning: invalid value encountered in less
  max_arg_R = np.choose(np.less(0.0, exp_arg_R), (0.0, exp_arg_R))


  inserted state between 0 and 0.0125 at 0.00625
  to improve acceptance rate from 0.3 to 0.3
Error with BAR. Using FEP.
Error with BAR. Using FEP.
  inserted state between 0 and 0.00625 at 0.003125
  to improve acceptance rate from 0.3 to 0.3
Error with BAR. Using FEP.
Error with BAR. Using FEP.
Error with BAR. Using FEP.
  inserted state between 0 and 0.003125 at 0.0015625
  to improve acceptance rate from 0.3 to 0.3
Error with BAR. Using FEP.
Error with BAR. Using FEP.
Error with BAR. Using FEP.
Error with BAR. Using FEP.
  inserted state between 0 and 0.0015625 at 0.00078125
  to improve acceptance rate from 0.3 to 0.3
Error with BAR. Using FEP.
Error with BAR. Using FEP.
Error with BAR. Using FEP.
Error with BAR. Using FEP.
Error with BAR. Using FEP.
  inserted state between 0 and 0.00078125 at 0.00039063
  to improve acceptance rate from 0.3 to 0.3
Error with BAR. Using FEP.
Error with BAR. Using FEP.
Error with BAR. Using FEP.
Error with BAR. Using FEP.
Error with BAR. Using FEP

/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/scipy/optimize/minpack.py:237: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


  calculated grid scaling free energy of 3504.88 RT using cycles 1 to 2
  wrote to f_RL.pkl.gz
  inserted state between 0 and 2.4414e-05 at 1.2207e-05
  to improve acceptance rate from 0.38154 to 0.58224
  saved dock progress and data

  completed cycle 3 in 20.54 s
  ExternalMC 1173/9250=0.13 (4.4 s); Sampler 1344/3750=0.36 (42.5 s); repX t 4.7 s
  saved dock progress and data

  calculated grid scaling free energy of 1855.76 RT using cycles 1 to 3
  wrote to f_RL.pkl.gz
  inserted state between 0.0125 and 0.025 at 0.01875
  to improve acceptance rate from 0.3854 to 0.66667
  inserted state between 0.01875 and 0.025 at 0.021875
  to improve acceptance rate from 0.33333 to 0.026082
  inserted state between 0.01875 and 0.021875 at 0.020313
  to improve acceptance rate from 0.026082 to 0.82206
  inserted state between 0.020313 and 0.021875 at 0.021094
  to improve acceptance rate from 0.12405 to 0.75822
  inserted state between 0.021094 and 0.021875 at 0.021484
  to improve acceptance ra